# cat-in-the-dat-ii(targetは連続量)

# import

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import os
from collections import deque

import codecs
import copy
from sklearn.metrics import confusion_matrix
import sys
from platform import python_version

import shutil
from sklearn.preprocessing import OneHotEncoder
import csv
from platform import python_version

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score



# environment

In [ ]:
print(python_version())

In [ ]:
sys.getdefaultencoding()

In [ ]:
%autosave 0

In [ ]:
#最大表示列数の指定（ここでは2000列を指定）
pd.set_option('display.max_columns', 2000)

In [ ]:
#最大表示行数の指定（ここでは200行を指定）
pd.set_option('display.max_rows', 200)

# definition

In [ ]:
### customize define
FEATURE_NUM = 90

CV_VAL = 10


In [ ]:
read_train_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/house_prices_ART/input/train.csv"

read_test_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/house_prices_ART/input/test.csv"

output_dir_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/house_prices_ART/output"

output_file_name = "submission.csv"

drop_col_name_list =[]

# 連続量
float_conv_col_name_list = [
    "LotFrontage",
    "LotArea",
    "YearBuilt",
    "YearRemodAdd",
    "MasVnrArea",
    "BsmtFinSF1",
    "BsmtFinSF2",
    "BsmtUnfSF",
    "TotalBsmtSF",
    "1stFlrSF",
    "2ndFlrSF",
    "LowQualFinSF",
    "GrLivArea",
    "GarageYrBlt",
    "GarageCars",
    "GarageArea",
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "3SsnPorch",
    "ScreenPorch",
    "PoolArea",
    "MiscVal",
    "MoSold",
    "YrSold"
]

# ↓にない連続量は0で初期化する。
float_nan_conv_dic ={
    # name: [min, max]
    "YearBuilt": [1872,  2010],
    "YearRemodAdd": [1950, 2010],
    "GarageYrBlt": [1900, 2010],
    "YrSold": [2006, 2010],
}

id_col_name="Id"
target_col_name = "SalePrice"

RANDOM_STATE_VAL = 0


# function

In [ ]:
def dispalyDf(readDf, dispFlag, row_num, dispLabel):
    print("========================= {} =========================".format(dispLabel))
    row_num , col_num = np.shape(readDf)
    print("row_num = {}, col_num = {}".format(row_num, col_num))
    if dispFlag:
        display(readDf.head(row_num))
    print("++++++ None Count All++++++")
    print(readDf.isnull().values.sum())
    print("++++++ None Count ++++++")
    print(readDf.isnull().sum())
    print("++++++ Col Type ++++++")
    print(readDf.dtypes)
#     print("++++++ Unique By Col（Noneは含まないので注意） ++++++")
#     for col_name in readDf.columns.tolist():
#         print("{} : {}".format(col_name, readDf[col_name].nunique()))
    

    

In [ ]:
def chkNum(expect_label, expect_num, act_label, act_num):
    if expect_num != act_num:
        raise Exception("Num Error !! expect({}): {}, act({}) : {}"
                        .format(expect_label, expect_num, act_label, act_num))
    

In [ ]:
def standardScalerDf(inputDf):
    row_num, col_num = np.shape(inputDf)
    
    # 平準化
    scaler = StandardScaler()
    val_array = scaler.fit_transform(np.array(inputDf.values.tolist()))
    standardScDf = pd.DataFrame(val_array, dtype=np.float, columns = inputDf.columns.tolist())
    
    sc_row_num, sc_col_num = np.shape(standardScDf)

    # 行数チェック
    chkNum("expect_row_num", row_num, "sc_row_num", sc_row_num)
    # 列数チェック
    chkNum("expect_col_num", col_num, "sc_col_num", sc_col_num)

    
    del val_array
    return standardScDf


In [ ]:
# test用のカラムのDataFrame
def getAdjustTestColDf(inputDf, train_col_list):
    not_enough_col_list = set(train_col_list) - set(inputDf.columns.tolist())
    print("# trainに有ってtestにないカラム名のリスト")
    print("{} : {}".format(len(not_enough_col_list), not_enough_col_list))
        
    dataLen = len(inputDf)
        
    retDf = inputDf.join(
        pd.DataFrame({col_name : [0]*dataLen  for col_name  in not_enough_col_list },
                     dtype=np.float, columns=not_enough_col_list)
    ).loc[:, train_col_list]
    
    # カラムリストが一致するかチェック
    if retDf.columns.tolist() != train_col_list:
        raise Exception("col_name_list not Match train and test.")

    # データ行数が一致するかチェック
    chkNum("dataLen", dataLen, "retDf", len(retDf))

    return retDf

In [ ]:
####################################
##### 次元削減(特徴量選択) を使用する
##### ↓特徴量選択(Ridge)
####################################
def getSelectorFeature(inputFeatureDf,y_Df,feature_num):
    
    # カラム名のリスト
    col_name_list = inputFeatureDf.columns.tolist()
    
    
    print("特徴量選択数 = ",feature_num)
    if(feature_num < 0):
        # デフォルト値
        print("n_features_to_select = None に設定")
        feature_num = None
    elif feature_num > len(col_name_list):
        raise Exception("feature_num({}) is Over colnum({})".format(feature_num, len(inputFeatureDf.columns.tolist())))
    
    ridge = Ridge(random_state=RANDOM_STATE_VAL)
    ridge.fit(inputFeatureDf, y_Df.values.ravel())
    
    
    coefDf = pd.DataFrame({
        "col_name": col_name_list,
        "coef_": ridge.coef_
    },dtype=object)
    
    # floatにカラム型変換
    coefDf["coef_"] = coefDf["coef_"].astype(np.float)

    # coef_のランクをつける
    coefDf['順位'] = (coefDf.rank(method='min',ascending=False)["coef_"]).astype('int')
    #coefDf = coefDf.sort_values('順位', ascending=True)

    
    coefDf = coefDf[coefDf["順位"] <= feature_num]
    
    inputFeatureNewDf = inputFeatureDf.loc[:,  coefDf["col_name"].values.tolist() ]

    return inputFeatureNewDf


In [ ]:
# 前処理関数
def getBeforeProcessDf(inputDf, drop_col_name_list, float_conv_col_name_list, train_Flg, train_target_Df ,
                       random_state_val, feature_num, train_col_list):
    # inputをコピーしておく
    convDf = inputDf.copy()
    
    # 列削除
    convDf = convDf.drop(drop_col_name_list, axis=1)
    
    # カラム名リスト保持
    col_list = convDf.columns.tolist()

    # 連続値のカラムの欠損値補完
    float_conv_dic={}
    for col_name in float_conv_col_name_list:
        # floatにカラム型変換
        convDf[col_name] = convDf[col_name].astype(np.float)
        
        conv_list = \
        [(
            np.random.randint(
                float_nan_conv_dic[col_name][0],
                float_nan_conv_dic[col_name][1])
            if col_name in float_nan_conv_dic else 0
        if np.isnan(elem) else elem )
         for elem in convDf[col_name].values.tolist()]
        
        float_conv_dic[col_name]=conv_list
    ###
    
    float_conv_df = pd.DataFrame(float_conv_dic,dtype=np.float)
    
    #dispalyDf(float_conv_df, True, 5,  "float_conv_df")
    
    col_list_except_float = [col_name for col_name in col_list 
                             if col_name not in float_conv_col_name_list]
    
    del conv_list
    # float以外のDataFrameを抽出
    convDf = convDf.loc[:,col_list_except_float]
    
    # NaNも一つのカテゴリーとしてダミー変数化したい場合は、引数dummy_na=Trueとする。
    convDf = pd.get_dummies(convDf, dummy_na=True, columns=col_list_except_float)
    

    #  全ての特徴量
    allFeatureDf = convDf.join(float_conv_df)

    if train_Flg:
        # trainの場合
        # 標準化
        allFeatureDf =  standardScalerDf(allFeatureDf)

        # Ridge で特徴量選択を行う。
        retFeatureDf = getSelectorFeature(allFeatureDf,  train_target_Df, feature_num)
        
        
    else:
        # testの場合
        # カラムをtrainと合わせる
        testFeatureDf = getAdjustTestColDf(allFeatureDf, train_col_list)
        # 標準化
        retFeatureDf =  standardScalerDf(testFeatureDf)
        
    
    return retFeatureDf

In [ ]:
# モデル入手
def getModel(train_Df, y_train_Df, random_state_val, cv_val):
    ridge = Ridge(random_state=random_state_val)
    ridge.fit(train_Df, y_train_Df.values.ravel())

    cv_results = cross_val_score(ridge,
                             train_Df,
                             y_train_Df.values.ravel(),      #1次元のARRAY
                             cv=cv_val,
                             scoring='r2')
    print(cv_results)

    return ridge


In [ ]:
# 予測
def getPredictDf(model, test_Df, id_Df , target_col_name):
    return id_Df.join(
        pd.DataFrame(model.predict(test_Df), dtype=np.float, columns=[target_col_name]))

# process

In [ ]:
# ファイル読み込み(train)
train_read_Df = pd.read_csv(read_train_path,
                            header=0,
                            encoding="utf-8",
                            sep='\n',
                            delimiter=',',
                            engine='python',
                            dtype=object)


In [ ]:
# ファイル読み込み(test)
test_read_Df = pd.read_csv(read_test_path,
                           header=0,
                           encoding="utf-8",
                           sep='\n',
                           delimiter=',',
                           engine='python',
                           dtype=object)


In [ ]:
######################################

In [ ]:
trainDf = train_read_Df.copy()
trainDf[target_col_name]=trainDf[target_col_name].astype(np.float)
testDf = test_read_Df.copy()

In [ ]:
dispalyDf(trainDf, True, 5,  "trainDf")

In [ ]:
dispalyDf(testDf, True, 5,  "testDf")

In [ ]:
# カラムの差分
set(trainDf.columns.tolist()) - set(testDf.columns.tolist())

In [ ]:
#################

In [ ]:
# train:データのDataFrameと、ターゲットのデータフレームに分ける。
train_target_Df = trainDf.loc[:,[target_col_name]]
trainDf = trainDf.loc[:,[col_name for col_name in trainDf.columns.tolist() if col_name not in [id_col_name,target_col_name]]]

In [ ]:
#dispalyDf(train_target_Df, True, 5,  "train_target_Df")

In [ ]:
#dispalyDf(trainDf, True, 5,  "trainDf")

In [ ]:
# test:idとデータのDataFrameを抽出。
test_id_Df = testDf.loc[:,[id_col_name]]
testDf = testDf.loc[:,[col_name for col_name in testDf.columns.tolist() if col_name != id_col_name]]

In [ ]:
#dispalyDf(test_id_Df, True, 5,  "test_id_Df")

In [ ]:
#dispalyDf(testDf, True, 5,  "testDf")

In [ ]:
#### train のデータ前処理
train_new_Df = getBeforeProcessDf(
    trainDf, drop_col_name_list, float_conv_col_name_list, True , train_target_Df,
    RANDOM_STATE_VAL, FEATURE_NUM, None)


In [ ]:
dispalyDf(train_new_Df, True, 5,  "train_new_Df")

In [ ]:
#### test のデータ前処理
test_new_Df= getBeforeProcessDf(
    testDf, drop_col_name_list, float_conv_col_name_list, False , None,
    None, None, train_new_Df.columns.tolist())


In [ ]:
dispalyDf(test_new_Df, True, 5,  "test_new_Df")

In [ ]:
###########

In [ ]:
# モデル入手
model = getModel(train_new_Df, train_target_Df, RANDOM_STATE_VAL, CV_VAL)

In [ ]:
# 予測
resultDf = getPredictDf(model, test_new_Df, test_id_Df , target_col_name)


In [ ]:
dispalyDf(resultDf, True, 5,  "resultDf")

In [ ]:
# ディレクトリなければ作る
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path) 

In [ ]:
ouput_file_path = os.path.join(output_dir_path, output_file_name)
print(ouput_file_path)

In [ ]:
# ファイル出力
resultDf.to_csv(ouput_file_path,
               sep=",",
                header=True,
                index=False,
                mode="w",
                encoding="utf-8",
                line_terminator="\n"
               )


In [ ]:
raise Exception("EEEEEEEEEEEEEEEEEEEEEE")